Notebook 8: Advanced Analytics - Leading Indicators & Fleet Age
* Goal: To create advanced visualizations that act as leading indicators for future demand and analyze the risk of demand destruction from aging reactors.

Block 1: Imports & Load Data

We'll import our standard libraries, plus datetime to calculate reactor ages. We'll load the pris_clean.csv (which we need for detailed analysis) and the forecast_data.csv and global_pipeline_ts.csv for our reality check plot.

In [89]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from datetime import datetime

pio.templates.default = "plotly_dark"

# Load the granular data
df_clean = pd.read_csv('pris_clean.csv')
# Load data for forecast-vs-pipeline plot
df_forecast = pd.read_csv('forecast_data.csv')
df_global_pipeline = pd.read_parquet('global_pipeline_ts.parquet')
# Load for getting top countries
df_national_demand = pd.read_csv('national_demand_ts.csv')


print("Loaded clean data, forecast data, and pipeline data.")

Loaded clean data, forecast data, and pipeline data.


Block 2: Visualization 1 - Net Demand Change (New Builds vs. Shutdowns)

This chart visualizes the "churn" in the global fleet. We'll find the first year a reactor came online ("New Build") and its first "Shutdown" year.

In [90]:
print("Generating Viz 1: Net Demand Change (Churn)...")

# Filter for valid years
df_clean_filtered = df_clean[df_clean['Year'] > 1950].copy()

# 1. Find all new builds (using Grid Connection as the "online" date)
df_clean_filtered['Grid Connection Date'] = pd.to_datetime(df_clean_filtered['Grid Connection Date'])
# Find the first connection record for each reactor
new_builds = df_clean_filtered.sort_values('Grid Connection Date').drop_duplicates('Reactor name')
new_builds['Connection Year'] = new_builds['Grid Connection Date'].dt.year
# Sum the capacity by connection year
new_capacity_by_year = new_builds.groupby('Connection Year')['Thermal Capacity, MWt'].sum().reset_index()

# 2. Find all shutdowns
# Find the first year a reactor is listed as "Permanent Shutdown"
shutdowns = df_clean_filtered[df_clean_filtered['Status'] == 'Permanent Shutdown'].sort_values('Year').drop_duplicates('Reactor name')
shutdown_capacity_by_year = shutdowns.groupby('Year')['Thermal Capacity, MWt'].sum().reset_index()

# 3. Merge them
churn_data = pd.merge(
    new_capacity_by_year.rename(columns={'Connection Year': 'Year', 'Thermal Capacity, MWt': 'New Capacity'}),
    shutdown_capacity_by_year.rename(columns={'Thermal Capacity, MWt': 'Shutdown Capacity'}),
    on='Year',
    how='outer'
).fillna(0)

# 4. Plot
fig_churn = go.Figure()
fig_churn.add_trace(go.Bar(
    x=churn_data['Year'],
    y=churn_data['New Capacity'],
    name='New Capacity Online',
    marker_color='green'
))
fig_churn.add_trace(go.Bar(
    x=churn_data['Year'],
    y=-churn_data['Shutdown Capacity'], # Plot as negative
    name='Capacity Shutdown',
    marker_color='red'
))
fig_churn.update_layout(
    barmode='relative',
    title='Net Thermal Capacity Change (New Builds vs. Shutdowns)',
    xaxis_title='Year',
    yaxis_title='Thermal Capacity (MWt)',
    legend_title='Event'
)
fig_churn.show()

Generating Viz 1: Net Demand Change (Churn)...


Block 3: Visualization 2 - Fleet Age Profile by Country (Risk Analysis)

This plot shows the age distribution of the current "Operational" fleet for the top countries, revealing shutdown risks.

In [91]:
print("\nGenerating Viz 2: Fleet Age Profile by Country...")

# 1. Get the current year
current_year = datetime.now().year

# 2. Get the current operational fleet
# We sort by Year and take the *last* record for each reactor
current_fleet = df_clean_filtered.sort_values('Year').drop_duplicates('Reactor name', keep='last')
current_operational_fleet = current_fleet[current_fleet['Status'] == 'Operational'].copy()

# 3. Calculate Age
current_operational_fleet['Grid Connection Date'] = pd.to_datetime(current_operational_fleet['Grid Connection Date'])
current_operational_fleet['Age'] = current_year - current_operational_fleet['Grid Connection Date'].dt.year

# 4. Get Top 10 countries (from our existing file)
latest_year = df_national_demand['Year'].max()
top_10_countries = df_national_demand[
    df_national_demand['Year'] == latest_year
].nlargest(10, 'Total Thermal Capacity, MWt')['Country'].tolist()

# 5. Filter fleet data for these countries
age_data_top_10 = current_operational_fleet[current_operational_fleet['Country'].isin(top_10_countries)]

# 6. Plot as a box plot
fig_age = px.box(
    age_data_top_10,
    x='Country',
    y='Age',
    color='Country',
    title='Fleet Age Distribution for Top 10 Countries (Operational Reactors)',
    points='all' # Show all reactors as dots
)
fig_age.update_layout(
    yaxis_title='Reactor Age (Years)',
    xaxis_title='Country'
)
fig_age.show()


Generating Viz 2: Fleet Age Profile by Country...


Block 4: Visualization 3 - Construction Pipeline vs. Forecast (Reality Check)

This dual-axis chart compares our statistical forecast (what could happen) with the "Under Construction" pipeline (what is happening).

In [92]:
print("\nGenerating Viz 3: Construction Pipeline vs. Statistical Forecast...")

# 1. Prepare forecast data
df_forecast['ds'] = pd.to_datetime(df_forecast['ds'])

# 2. Prepare pipeline data
df_construction_ts = df_global_pipeline[
    (df_global_pipeline['Status'] == 'Under Construction') &
    (df_global_pipeline['Year'] > 1950)
].copy()
# Convert Year to datetime to align axes
df_construction_ts['ds'] = pd.to_datetime(df_construction_ts['Year'], format='%Y')

# 3. Create the dual-axis chart
fig_reality_check = make_subplots(specs=[[{"secondary_y": True}]])

# 4. Add Forecast Traces (Left Y-Axis)
fig_reality_check.add_trace(go.Scatter(
    x=df_forecast['ds'],
    y=df_forecast['yhat'],
    mode='lines',
    name='Forecast Capacity (yhat)',
    line=dict(color='cyan')
), secondary_y=False)
fig_reality_check.add_trace(go.Scatter(
    x=df_forecast['ds'],
    y=df_forecast['yhat_lower'],
    mode='lines',
    name='Forecast Lower Bound',
    line=dict(width=0, color='gray'),
    showlegend=False
), secondary_y=False)
fig_reality_check.add_trace(go.Scatter(
    x=df_forecast['ds'],
    y=df_forecast['yhat_upper'],
    mode='lines',
    name='Forecast Upper Bound',
    line=dict(width=0, color='gray'),
    fill='tonexty',
    fillcolor='rgba(0,176,246,0.2)',
    showlegend=False
), secondary_y=False)

# 5. Add Construction Trace (Right Y-Axis)
fig_reality_check.add_trace(go.Scatter(
    x=df_construction_ts['ds'],
    y=df_construction_ts['Reactor Count'],
    mode='lines',
    name='Reactors "Under Construction"',
    line=dict(color='orange', dash='dash')
), secondary_y=True)

# 6. Set titles
fig_reality_check.update_layout(
    title='Forecast vs. Reality: Global Capacity Forecast vs. Construction Pipeline'
)
fig_reality_check.update_yaxes(title_text='<b>Forecast Thermal Capacity (MWt)</b>', secondary_y=False)
fig_reality_check.update_yaxes(title_text='<b>Reactors Under Construction (Count)</b>', secondary_y=True)

fig_reality_check.show()


Generating Viz 3: Construction Pipeline vs. Statistical Forecast...


Block 5: Visualization 4 - Geospatial Demand Map (Choropleth)

This map gives us an immediate "at-a-glance" view of where global demand is concentrated.

In [93]:
print("\nGenerating Viz 4: Geospatial Demand Map...")

# 1. Get data for the latest year
latest_year = df_national_demand['Year'].max()
df_map_data = df_national_demand[df_national_demand['Year'] == latest_year]

# 2. Create the choropleth map
fig_map = px.choropleth(
    df_map_data,
    locations="Country",
    locationmode="country names",
    color="Total Thermal Capacity, MWt",
    hover_name="Country",
    color_continuous_scale=px.colors.sequential.Plasma,
    title=f"Global Operational Thermal Capacity by Country ({latest_year})"
)

fig_map.show()
print("\n--- Notebook 8 complete. ---")


Generating Viz 4: Geospatial Demand Map...



--- Notebook 8 complete. ---


Block 6: Visualization 5 - New Capacity by Technology (Leading Indicator)

This stacked bar chart is a powerful leading indicator. It shows which reactor technologies are being chosen for new builds over time. This can help predict future trends, such as a shift to Small Modular Reactors (SMRs) or next-generation designs, which may have different fuel requirements.

In [94]:
print("\nGenerating Viz 5: New Capacity by Technology...")

# 1. Find all new builds (using Grid Connection as the "online" date)
# We re-use df_clean_filtered from the previous blocks
df_clean_filtered['Grid Connection Date'] = pd.to_datetime(df_clean_filtered['Grid Connection Date'])
new_builds = df_clean_filtered.sort_values('Grid Connection Date').drop_duplicates('Reactor name')
new_builds['Connection Year'] = new_builds['Grid Connection Date'].dt.year

# 2. Group by Connection Year AND Type, then sum capacity
new_capacity_by_tech = new_builds.groupby(['Connection Year', 'Type'])['Thermal Capacity, MWt'].sum().reset_index()

# 3. Plot as a stacked bar chart
fig_tech_builds = px.bar(
    new_capacity_by_tech[new_capacity_by_tech['Connection Year'] > 1950], # Filter year 0
    x='Connection Year',
    y='Thermal Capacity, MWt',
    color='Type',
    title='New Thermal Capacity Added by Reactor Type (Leading Tech Indicator)'
)
fig_tech_builds.update_layout(
    xaxis_title='Year of Grid Connection',
    yaxis_title='New Thermal Capacity (MWt)',
    legend_title='Reactor Type'
)
fig_tech_builds.show()


Generating Viz 5: New Capacity by Technology...


Block 7: Visualization 6 - Global Average Load Factor (Efficiency Trend)

This line chart analyzes the operational efficiency of the global fleet. A "load factor" is the ratio of the actual energy output over a period to the maximum possible output. A rising trend (like the one seen in the data) means the global fleet is becoming more efficient, generating more electricity (and consuming fuel more consistently) per reactor.

Note: To build this, we must re-load the original raw data file, as Load Factor Annual was not in our pris_clean.csv.

In [96]:
print("\nGenerating Viz 6: Global Average Load Factor...")

# 1. Load the original raw data file to get 'Load Factor Annual'
df_raw = pd.read_excel('Reactor Database - master(pris.iaea.xlsx')

# 2. Clean the necessary columns
df_load_factor = df_raw[['Year', 'Status', 'Load Factor Annual']].copy()
df_load_factor['Load Factor Annual'] = pd.to_numeric(df_load_factor['Load Factor Annual'], errors='coerce')

# 3. Filter for operational reactors and valid data
df_load_factor_op = df_load_factor[
    (df_load_factor['Status'] == 'Operational') &
    (df_load_factor['Year'] > 1950) &
    (df_load_factor['Load Factor Annual'].notna()) &
    (df_load_factor['Load Factor Annual'] > 0) # A 0% LF isn't useful for the average
]

# 4. Calculate the average load factor for each year
avg_load_factor_ts = df_load_factor_op.groupby('Year')['Load Factor Annual'].mean().reset_index()

# 5. Plot as a line chart
fig_load_factor = px.line(
    avg_load_factor_ts,
    x='Year',
    y='Load Factor Annual',
    title='Global Average Annual Load Factor (Operational Fleet Efficiency)'
)
fig_load_factor.update_layout(
    xaxis_title='Year',
    yaxis_title='Average Annual Load Factor (%)',
    yaxis_ticksuffix='%'
)

# 6. Add a 10-year rolling average to smooth the line and see the long-term trend
avg_load_factor_ts['10-Year Rolling Avg'] = avg_load_factor_ts['Load Factor Annual'].rolling(10, center=True).mean()
fig_load_factor.add_trace(go.Scatter(
    x=avg_load_factor_ts['Year'],
    y=avg_load_factor_ts['10-Year Rolling Avg'],
    mode='lines',
    name='10-Year Rolling Avg',
    line=dict(color='red', dash='dash')
))
fig_load_factor.show()
print("\n--- Notebook 8 complete. ---")


Generating Viz 6: Global Average Load Factor...



--- Notebook 8 complete. ---
